In [7]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [8]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [9]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [13]:
Poly_Ring.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [14]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i_rad3*c})

CPU times: user 3.84 s, sys: 19.4 ms, total: 3.86 s
Wall time: 3.87 s


In [6]:
# Find simmetries

In [7]:
%time adm_SE9 = SE9.find_admissible_projectivities()
len(adm_SE9)

CPU times: user 4.38 s, sys: 234 ms, total: 4.62 s
Wall time: 53.8 s


1296

In [8]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)
len(simm_SE9)

CPU times: user 2.1 s, sys: 173 ms, total: 2.27 s
Wall time: 7.18 s


54

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_points_perms = []
for proj in simm_SE9:
    Eck_points_perms.append(Permutation(SE9.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(18, '(C3 x C3) : C2')

In [11]:
# Study how the simmetries permute the 27 lines

In [12]:
lines_perms = []
for simm in simm_SE9:
    lines_perms.append(Permutation(SE9.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(54, '((C3 x C3) : C3) : C2')

In [13]:
all_subgroups = lines_perms_group.subgroups()

In [14]:
# H is the subgroup of elements of order 3
H = [el for el in lines_perms_group.normal_subgroups() if el.order()==27][0]
list(set([el.order() for el in H if el.cycle_tuples() != []])) == [3], H.structure_description()

(True, '(C3 x C3) : C3')

In [15]:
# The center is C3
Z_H = H.center()
M = Z_H.gens_small()[0]
Z_H.structure_description(), M

('C3',
 (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21))

In [16]:
len([el for el in all_subgroups if el.is_isomorphic(H.quotient(Z_H))])

4

In [17]:
H.quotient(Z_H)

Permutation Group with generators [(1,2,4)(3,5,7)(6,8,9), (1,3,6)(2,5,8)(4,7,9)]

In [18]:
H_over_Z_H = [el for el in all_subgroups if el.is_isomorphic(H.quotient(Z_H))][0]
[N, P] = H_over_Z_H.gens_small()
H_over_Z_H.structure_description()

'C3 x C3'

In [19]:
H.structure_description()

'(C3 x C3) : C3'

In [20]:
H.minimal_generating_set()

[(1,9,14)(2,19,10)(3,13,4)(5,21,17)(6,20,16)(7,8,22)(11,24,26)(12,23,25)(15,18,27),
 (1,8,13)(2,16,4)(3,27,17)(5,25,10)(6,23,14)(7,11,19)(9,21,26)(12,18,22)(15,20,24)]

In [21]:
P1 = [el for el in H if el not in PermutationGroup([M,N])][0]

In [22]:
PermutationGroup([M,N, P1]).order()

27

In [23]:
PermutationGroup([M,N,P1]).is_isomorphic(H)

True

In [24]:
C2 = [el for el in all_subgroups if el.is_isomorphic(lines_perms_group.quotient(H))][0]
A = C2.gens_small()[0]

In [25]:
[el.order() for el in H.normal_subgroups()]

[27, 9, 9, 9, 9, 3, 1]

In [26]:
gens = H.gens()+[A]
gens_indices = [lines_perms.index(gen) for gen in gens]
matrix_gens = [simm_SE9[index] for index in gens_indices]

In [27]:
# There are twelve lines through 3 Eckardt points, and they are all coplanar

In [29]:
lines_three_eck = []
lines_labels = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9.eckardt_points[i]
            point2 = SE9.eckardt_points[j]
            point3 = SE9.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                lines_labels.append([i+1,j+1,k+1])
                points = [point1, point2]
                planes = get_two_planes_containing_line(points)
                lines_three_eck.append(Line(planes, points, line12_pl))
print(lines_labels)

[[1, 2, 8], [1, 3, 9], [1, 4, 6], [1, 5, 7], [2, 3, 5], [2, 4, 9], [2, 6, 7], [3, 4, 7], [3, 6, 8], [4, 5, 8], [5, 6, 9], [7, 8, 9]]


In [30]:
False in [[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck]

False

In [31]:
lines_three_eck[9].get_plane_containing_another_incident_line(lines_three_eck[6])

x + (-1/4*i_rad3 - 1/4)*y + (1/4*i_rad3 + 1/4)*z

In [32]:
# Study how the 12 lines are permuted

In [33]:
for gen in matrix_gens:
    print(Permutation([lines_three_eck.index(line.apply_proj(gen))+1 for line in lines_three_eck]).to_permutation_group_element())

(1,11,8)(2,10,7)(3,5,12)
(2,7,10)(3,5,12)(4,6,9)
()
(5,12)(6,9)(7,10)(8,11)


In [34]:
[Eck_points_perms[gen_index] for gen_index in gens_indices]

[(1,7,5)(2,4,9)(3,6,8), (1,8,2)(3,4,7)(5,6,9), (), (2,8)(3,9)(4,6)(5,7)]

In [35]:
keys = list(SE9.cl_lines.keys())
print(SE9.eckardt_points_labels)
print()
for gen in gens:
    labels = from_perm_to_labels(gen)
    print([[keys[labels.index(label)] for label in Eck_label] for Eck_label in SE9.eckardt_points_labels])
    print()

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['E3', 'G5', 'F35'], ['E4', 'G6', 'F46'], ['E5', 'G2', 'F25'], ['E6', 'G1', 'F16'], ['F12', 'F36', 'F45'], ['F13', 'F24', 'F56'], ['F15', 'F26', 'F34']]

[['F12', 'F45', 'F36'], ['E4', 'F46', 'G6'], ['F16', 'G1', 'E6'], ['F15', 'F34', 'F26'], ['G4', 'E1', 'F14'], ['F13', 'F24', 'F56'], ['G2', 'F25', 'E5'], ['F35', 'G5', 'E3'], ['E2', 'G3', 'F23']]

[['F13', 'F24', 'F56'], ['G4', 'E1', 'F14'], ['E4', 'F46', 'G6'], ['F12', 'F45', 'F36'], ['F16', 'G1', 'E6'], ['F15', 'F34', 'F26'], ['E3', 'G5', 'F35'], ['G3', 'E2', 'F23'], ['F25', 'E5', 'G2']]

[['F14', 'E1', 'G4'], ['G3', 'F23', 'E2'], ['G5', 'F35', 'E3'], ['F46', 'E4', 'G6'], ['G2', 'F25', 'E5'], ['F16', 'E6', 'G1'], ['F36', 'F45', 'F12'], ['F56', 'F13', 'F24'], ['F26', 'F34', 'F15']]

[['E1', 'G4', 'F14'], ['F24', 'F13', 'F56'], ['F34', 'F15', 'F26'], ['G1', 'F16', 'E6'], ['F45', 'F12', 'F36'], ['F46', 'E4', 'G6'], ['G2', 'F25', 'E5'], ['G3', 'E2', 'F23'], ['G5', 'F35', 'E3']]



In [36]:
gens_lines = [lines_perms[index] for index in gens_indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('G2', 'E5', 'F15', 'F26', 'F56', 'F45'),
  ('G3', 'E2', 'F24', 'F13', 'F12', 'F26'),
  ('G4', 'F14', 'F23', 'E2', 'F35', 'F25'),
  ('E1', 'G4', 'F24', 'F13', 'F34', 'F45')],
 [(1,8,13)(2,16,4)(3,27,17)(5,25,10)(6,23,14)(7,11,19)(9,21,26)(12,18,22)(15,20,24),
  (1,9,14)(2,19,10)(3,13,4)(5,21,17)(6,20,16)(7,8,22)(11,24,26)(12,23,25)(15,18,27),
  (1,10,15)(2,18,9)(3,23,11)(4,12,26)(5,20,8)(6,7,17)(13,25,24)(14,19,27)(16,22,21),
  (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)])